# Object Classification with Faster-RCNN

This notebook demonstrates training a YOLOv8 model for object classification, specifically for car detection with multiple classes. It covers data loading, model training, evaluation, and inference.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# === PATH DEFINITIONS ===
import os # Import os here as it's used immediately

DRIVE_MOUNT_POINT = '/content/drive'
BASE_PROJECT_PATH = '/content/drive/MyDrive/Projects/Car Detection v2/'
DATASET_BASE_PATH = os.path.join(BASE_PROJECT_PATH, 'Single Class Data')

# Training data paths
TRAIN_IMAGES_PATH = os.path.join(DATASET_BASE_PATH, 'train/images/')
TRAIN_LABELS_PATH = os.path.join(DATASET_BASE_PATH, 'train/labels/')

# Validation data paths
VALID_IMAGES_PATH = os.path.join(DATASET_BASE_PATH, 'valid/images/')
VALID_LABELS_PATH = os.path.join(DATASET_BASE_PATH, 'valid/labels/')

# Test data path
TEST_IMAGES_DIR = os.path.join(DATASET_BASE_PATH, 'test/images')

# Dataset configuration file
DATASET_YAML_PATH = os.path.join(DATASET_BASE_PATH, 'data.yaml')

# Model training output paths
MODEL_OUTPUT_BASE_PATH = os.path.join(BASE_PROJECT_PATH, 'Colab Env/Model History')
EXPERIMENT_NAME = 'singleclass_colab_Faster-RCNN' # Choose a descriptive name
FULL_EXPERIMENT_PATH = os.path.join(MODEL_OUTPUT_BASE_PATH, EXPERIMENT_NAME)

# Paths to specific files generated during training
BEST_WEIGHTS_PATH = os.path.join(FULL_EXPERIMENT_PATH, 'weights/best.pt')
RESULTS_CSV_PATH = os.path.join(FULL_EXPERIMENT_PATH, 'results.csv')
CONFUSION_MATRIX_IMG_PATH = os.path.join(FULL_EXPERIMENT_PATH, 'confusion_matrix.png')

# Paths to video test
VIDEO_TEST = os.path.join(BASE_PROJECT_PATH, 'Video Test')
VIDEO_TEST_PREDICTION = os.path.join(BASE_PROJECT_PATH, 'Video Test Prediction Colab Env')

print(f"Base Project Path: {BASE_PROJECT_PATH}")
print(f"Dataset Path: {DATASET_BASE_PATH}")
print(f"Dataset YAML Path: {DATASET_YAML_PATH}")
print(f"Model Output Path: {FULL_EXPERIMENT_PATH}")
print(f"Best Weights will be saved to: {BEST_WEIGHTS_PATH}")

Base Project Path: /content/drive/MyDrive/Projects/Car Detection v2/
Dataset Path: /content/drive/MyDrive/Projects/Car Detection v2/Single Class Data
Dataset YAML Path: /content/drive/MyDrive/Projects/Car Detection v2/Single Class Data/data.yaml
Model Output Path: /content/drive/MyDrive/Projects/Car Detection v2/Colab Env/Model History/singleclass_colab_Faster-RCNN
Best Weights will be saved to: /content/drive/MyDrive/Projects/Car Detection v2/Colab Env/Model History/singleclass_colab_Faster-RCNN/weights/best.pt


# Install Depedencies and Import Libraries

In [ ]:
!pip install torch torchvision opencv-python-headless

# === Deep Learning Model ===
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
import cv2
import numpy as np
import os
import time

# === Visualization ===
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin